# Arm swing quantification
This notebook shows the process of segmenting and quantifying the arm swing of predicted gait without other arm activities. The inputs are arm swing features (output from 4.extract_features_arm_swing.ipynb) and arm swing predictions (output from 5.arm_swing_detection.ipynb), and the output is a dataframe of arm swing features aggregated per segment (i.e., contiguous samples of predicted arm swing).

## Modules

In [1]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import pandas as pd
import tsdf

from dbpd.feature_extraction import *
from dbpd.windowing import *
from dbpd.quantification import aggregate_segments

## Constants

In [2]:
# Cell has the tag 'parameters'
path_to_data = '../../../tests/data'

output_path = os.path.join(path_to_data, '5.quantification', 'gait')

In [3]:
# paths and files
path_to_feature_input = os.path.join(path_to_data, '3.extracted_features', 'gait')
path_to_prediction_input = os.path.join(path_to_data, '4.predictions', 'gait')


meta_filename = 'arm_swing_meta.json'
time_filename = 'arm_swing_time.bin'
values_filename = 'arm_swing_values.bin'

pred_arm_swing_colname = 'pred_arm_swing'

window_length_s = 3
window_step_size = 0.75
segment_gap_s = 3
min_segment_length_s = 3

## Load data

### Features

In [4]:
metadata_dict = tsdf.load_metadata_from_path(os.path.join(path_to_feature_input, meta_filename))
metadata_time = metadata_dict[time_filename]
metadata_samples = metadata_dict[values_filename]

df_features = tsdf.load_dataframe_from_binaries([metadata_time, metadata_samples], tsdf.constants.ConcatenationType.columns)

df_features.head(2)

,time,angle_perc_power,range_of_motion,forward_peak_ang_vel_mean,forward_peak_ang_vel_std,backward_peak_ang_vel_mean,backward_peak_ang_vel_std,std_norm_acc,grav_accelerometer_x_mean,grav_accelerometer_x_std,...,cc_7_gyroscope,cc_8_gyroscope,cc_9_gyroscope,cc_10_gyroscope,cc_11_gyroscope,cc_12_gyroscope,cc_13_gyroscope,cc_14_gyroscope,cc_15_gyroscope,cc_16_gyroscope
0,0.00,0.933953,2.875672,8.193249,3.292503,4.938885,0.0,0.154163,-0.254277,0.201148,...,0.911306,3.910365,2.699518,2.580446,1.602991,0.556265,-1.046647,0.541998,-0.346748,0.365782
1,0.75,0.992147,4.295439,3.288568,1.612178,4.401343,0.0,0.160148,-0.395660,0.176619,...,2.608440,4.632687,2.083540,2.834911,1.474186,-0.073377,-2.122460,0.373824,1.433426,1.711163


### Predictions

In [5]:
metadata_dict = tsdf.load_metadata_from_path(os.path.join(path_to_prediction_input, meta_filename))
metadata_time = metadata_dict[time_filename]
metadata_samples = metadata_dict[values_filename]

df_predictions = tsdf.load_dataframe_from_binaries([metadata_time, metadata_samples], tsdf.constants.ConcatenationType.columns)

df_predictions.head(2)

,time,pred_arm_swing
0,0.00,1
1,0.75,1


### Validate

In [6]:
# dataframes have same length
assert df_features.shape[0] == df_predictions.shape[0]

# dataframes have same time column
assert df_features['time'].equals(df_predictions['time'])

## Prepare data

In [7]:
# subset features
l_feature_cols = ['time', 'range_of_motion', 'forward_peak_ang_vel_mean', 'backward_peak_ang_vel_mean']
df_features = df_features[l_feature_cols]

# concatenate features and predictions
df = pd.concat([df_features, df_predictions[pred_arm_swing_colname]], axis=1)

# temporarily for testing: manually determine predictions
df[pred_arm_swing_colname] = np.concatenate([np.repeat([1], df.shape[0]//3), np.repeat([0], df.shape[0]//3), np.repeat([1], df.shape[0] - 2*df.shape[0]//3)], axis=0)

# keep only predicted arm swing
df_arm_swing = df.loc[df[pred_arm_swing_colname]==1].copy().reset_index(drop=True)

del df

# create peak angular velocity
df_arm_swing.loc[:, 'peak_ang_vel'] = df_arm_swing.loc[:, ['forward_peak_ang_vel_mean', 'backward_peak_ang_vel_mean']].mean(axis=1)
df_arm_swing = df_arm_swing.drop(columns=['forward_peak_ang_vel_mean', 'backward_peak_ang_vel_mean'])

### Segmenting

In [8]:
df_arm_swing = create_segments(
    df=df_arm_swing,
    time_colname='time',
    segment_nr_colname='segment_nr',
    minimum_gap_s=segment_gap_s
)

df_arm_swing = discard_segments(
    df=df_arm_swing,
    time_colname='time',
    segment_nr_colname='segment_nr',
    minimum_segment_length_s=min_segment_length_s
)

## Quantify arm swing

In [9]:
df_aggregates = aggregate_segments(
    df=df_arm_swing,
    time_colname='time',
    segment_nr_colname='segment_nr',
    window_step_size_s=window_step_size,
    l_metrics=['range_of_motion', 'peak_ang_vel'],
    l_aggregates=['median'],
    l_quantiles=[0.95]
)

df_aggregates.head(2)

,segment_nr,range_of_motion_median,range_of_motion_quantile_95,peak_ang_vel_median,peak_ang_vel_quantile_95,time,segment_duration_s
0,1,0.000000,12.439385,0.000000,50.282076,0.00,160.5
1,2,4.657872,16.370933,10.375275,52.173041,567.43,160.5


In [10]:
df_aggregates['segment_duration_ms'] = df_aggregates['segment_duration_s'] * 1000

## Store data

In [11]:
df_aggregates = df_aggregates.drop(columns=['segment_nr'])

In [12]:
metadata_samples.__setattr__('file_name', 'arm_swing_values.bin')
metadata_samples.__setattr__('file_dir_path', output_path)
metadata_time.__setattr__('file_name', 'arm_swing_time.bin')
metadata_time.__setattr__('file_dir_path', output_path)

metadata_samples.__setattr__('channels', ['range_of_motion_median', 'range_of_motion_quantile_95',
                                          'peak_ang_vel_median', 'peak_ang_vel_quantile_95'])
metadata_samples.__setattr__('units', ['deg', 'deg', 'deg/s', 'deg/s'])
metadata_samples.__setattr__('data_type', np.float32)
metadata_samples.__setattr__('bits', 32)

metadata_time.__setattr__('channels', ['time', 'segment_duration_ms'])
metadata_time.__setattr__('units', ['relative_time_ms', 'ms'])
metadata_time.__setattr__('data_type', np.int32)
metadata_time.__setattr__('bits', 32)

In [13]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

# store binaries and metadata
tsdf.write_dataframe_to_binaries(output_path, df_aggregates, [metadata_time, metadata_samples])
tsdf.write_metadata([metadata_time, metadata_samples], 'arm_swing_meta.json')